In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 46.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 53.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 59.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 27.4 MB/s eta 0:00:00


In [3]:
from transformers import BartTokenizer

model_name = 'facebook/bart-base'

tokenizer = BartTokenizer.from_pretrained(model_name)

In [4]:
import pandas as pd

dataframe = pd.read_parquet('/content/drive/MyDrive/toxicity.parquet')
dataframe.head()

,text,labels
0,"If Alkar is flooding her with psychic waste, t...",0.014195
1,"if Alkar floods her with her mental waste, it ...",0.981983
2,Now you're getting nasty.,0.065473
3,you're becoming disgusting.,0.999039
4,"Well, we could spare your life, for one.",0.213313


In [5]:
texts = dataframe.text.values
labels = dataframe.labels.values

In [6]:
def get_tokenized(text: str):
    encoded_dict = tokenizer.encode_plus(
                        text,
                        add_special_tokens = True,
                        max_length = 512,
                        pad_to_max_length = True,
                        return_attention_mask = True,
                        truncation=True,
                        return_tensors = 'pt'
                        )
    return encoded_dict['input_ids']

In [7]:
input_ids = list(map(get_tokenized, texts))

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2614: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [8]:
import torch

input_ids = torch.cat(input_ids, dim=0)
labels = torch.from_numpy(labels).unsqueeze(1).float()

In [9]:
from torch.utils.data import TensorDataset, random_split

dataset = TensorDataset(input_ids, labels)

generator = torch.Generator().manual_seed(123)

train_dataset, validation_dataset, test_dataset = random_split(dataset, [0.8, 0.1, 0.1], generator=generator)

In [10]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 16

train_dataloader = DataLoader(
    train_dataset,
    sampler = RandomSampler(train_dataset, generator=generator),
    batch_size = batch_size
)

validation_dataloader = DataLoader(
    validation_dataset,
    sampler = SequentialSampler(validation_dataset),
    batch_size = batch_size
)

test_dataloader = DataLoader(
    test_dataset,
    sampler = SequentialSampler(test_dataset),
    batch_size = batch_size
)

In [11]:
from torch import nn
import torch.nn.functional as F


class ToxicityEvaluation(nn.Module):
    def __init__(self):
        super(ToxicityEvaluation, self).__init__()
        self.embedding = nn.Embedding(tokenizer.vocab_size, 100)
        self.conv1 = nn.Conv1d(100, 128, kernel_size=3)
        self.conv2 = nn.Conv1d(128, 256, kernel_size=3)
        self.fc1 = nn.Linear(256, 128)
        self.fc2 = nn.Linear(128, 1)

    def forward(self, x):
        x = self.embedding(x)
        x = x.permute(0, 2, 1)
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.max_pool1d(x, kernel_size=x.size(2))
        x = x.squeeze(2)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [12]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = ToxicityEvaluation().to(device)

In [13]:
from torch.optim import Adam

optimizer = Adam(model.parameters())

In [14]:
loss = nn.MSELoss()

In [17]:
import random
import numpy as np
from tqdm import tqdm

seed_val = 123

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)
best_loss = float('inf')

epochs = 10

for epoch in range(epochs):
    total_train_loss = 0

    train_loop = tqdm(
        train_dataloader, total=len(train_dataloader), desc=f"Epoch {epoch}"
    )

    model.train()

    for batch in train_loop:
        b_input_ids = batch[0].to(device)
        b_labels = batch[1].to(device)

        model.zero_grad()

        output = model(b_input_ids)

        _loss = loss(output, b_labels)

        total_train_loss += _loss.item()
        _loss.backward()

        optimizer.step()
        train_loop.set_postfix({"loss": _loss.item()})

    avg_train_loss = total_train_loss / len(train_dataloader)


    model.eval()
    total_eval_loss = 0
    nb_eval_steps = 0

    validation_loop = tqdm(validation_dataloader, total=len(validation_dataloader), desc="Val")

    for batch in validation_loop:
        b_input_ids = batch[0].to(device)
        b_labels = batch[1].to(device)

        with torch.no_grad():
            output = model(b_input_ids)

        _loss = loss(output, b_labels)

        total_eval_loss += _loss.item()

    avg_val_loss = total_eval_loss / len(validation_dataloader)
    if avg_val_loss > best_loss:
        torch.save(model.state_dict(), 'best.pt')


Val: 100%|██████████| 3612/3612 [00:05<00:00, 678.31it/s]


In [24]:
model(get_tokenized('you are so pretty person').to(device))

tensor([[0.2071]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [25]:
torch.save(model.state_dict(), 'best.pt')